# Recolectar datos a través de la API de Space X 🚀

<strong>Objetivos:</strong>
+ Solicitar datos a la API de Space X.
+ Limpiar los datos obtenidos.

---

## Importar librerías y definir funciones auxiliares

In [1]:
# Importar.
import requests
import pandas as pd
import numpy as np
import datetime
import json
from pandas.io.json import json_normalize

# Para imprimir todas las columnas de un DataFrame.
pd.set_option('display.max_columns', None)
# Para evitar que los textos se recorten.
pd.set_option('display.max_colwidth', None)

# Para ignorar Warnings.
import warnings
warnings.filterwarnings('ignore')

A continuación, definir una serie de funciones auxiliares para que la API extraiga información de sus datos de lanzamiento de cohetes a partir de IDs.

In [2]:
# Primero, definir listas / variables vacias
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [3]:
# De la columna 'rocket' (cohete) obtener los nombres de los boosters (propulsores).
'''
Un booster es una sección de un cohete diseñada para aumentar la velocidad y la altitud de una nave espacial. Está compuesto por un motor y un tanque de combustible y se usa para impulsar una nave durante los primeros minutos de su vuelo, permitiendo que alcance la velocidad y altitud necesarias para entrar en órbita.
'''
# A partir del dataset, usar la columna 'rocket' para llamar a la API y obtener el nombre de los boosters.
# Agregar los datos a la lista BoosterVersion.
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
            response = requests.get('https://api.spacexdata.com/v4/rockets/'+str(x)).json()
            BoosterVersion.append(response['name']) # <--

In [4]:
# De la columna 'launchpad', obtener el nombre del sitio de lanzamiento que se está utilizando, la longitud y la latitud.
# A partir del dataset, usar la columna 'launchpad' para llamar a la API y agregar los datos de interés a las listas específicadas.
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
            response = requests.get('https://api.spacexdata.com/v4/launchpads/'+str(x)).json()
            Longitude.append(response['longitude']) # <--
            Latitude.append(response['latitude']) # <--
            LaunchSite.append(response['name']) # <--

In [5]:
# De la columna 'payloads' (carga útil) obtener la masa de la carga útil y la órbita a la que se dirige.
# A partir del dataset, usar la columna 'payloads' para llamar a la API y agregar los datos de interés a las listas específicadas.
def getPayloadData(data):
    for load in data['payloads']:
       if load:
            response = requests.get('https://api.spacexdata.com/v4/payloads/'+load).json()
            PayloadMass.append(response['mass_kg']) # <--
            Orbit.append(response['orbit']) # <--

In [6]:
'''
De la columna 'cores' (núcleos) obtener el resultado del aterrizaje, el tipo de aterrizaje, el número de vuelos con ese núcleo, si se usaron gridfins, si se reutilizó el núcleo, si se usaron patas, la plataforma de aterrizaje utilizada, el bloque de el núcleo (se usa para separar la versión de los núcleos), la cantidad de veces que se ha reutilizado este núcleo específico y el número de serie del núcleo.
'''
# A partir del dataset, usar la columna 'cores' para llamar a la API y agregar los datos de interés a las listas específicadas.
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get('https://api.spacexdata.com/v4/cores/'+core['core']).json()
                Block.append(response['block']) # <--
                ReusedCount.append(response['reuse_count']) # <--
                Serial.append(response['serial']) # <--
            else:
                Block.append(None) # <--
                ReusedCount.append(None) # <--
                Serial.append(None) # <--
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type'])) # <--
            Flights.append(core['flight']) # <--
            GridFins.append(core['gridfins']) # <--
            Reused.append(core['reused']) # <--
            Legs.append(core['legs']) # <--
            LandingPad.append(core['landpad']) # <--

## Solicitar datos a la API

In [7]:
# Asignar la url de la API de Space X a una variable.
spacex_url = 'https://api.spacexdata.com/v4/launches/past'
# Hacer solicitud HTTP GET a la url para obtener los datos almacenados en esa ubicación,
# y asignar resultados a la variable response.
response = requests.get(spacex_url)

In [9]:
# Ver el contenido de la respuesta.
#print(response.content)

La respuesta contiene información masiva sobre los lanzamientos de Space X, pero todavía queda más información relevante que descubrir en este proyecto.

## Tareas

### Tarea 1: Solicitar y parsear los datos de lanzamiento de Space X. Utilizar la solicitud GET

In [10]:
# Checkear que la solicitud fue exitosa.
response.status_code

200

In [11]:
# Decodificar el contenido de la respuesta como un Json usando .json()
# y convertirlo en un DataFrame de Pandas usando .json_normalize()

# import json
# from pandas.io.json import json_normalize

data = pd.json_normalize(response.json())

In [12]:
# Mostrar las primeras 5 filas del DataFrame.
data.head()

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/94/f2/NN6Ph45r_o.png,https://images2.imgbox.com/5b/02/QcxHUb5V_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutdown'}]","Successful first stage burn and transition to second stage, maximum altitude 289 km, Premature engine shutdown at T+7 min 30 s, Failed to reach orbit, Failed to recover first stage",[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdaffd86e000604b32b,False,False,False,[],https://images2.imgbox.com/f9/4a/ZboXReNb_o.png,https://images2.imgbox.com/80/a2/bkWotCIS_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-rocket-fails-reach-orbit.html,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'residual stage-1 thrust led to collision between stage 1 and stage 2'}]",Residual stage 1 thrust led to collision between stage 1 and stage 2,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006eeb1e4]",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdbffd86e000604b32c,False,False,False,[],https://images2.imgbox.com/6c/cb/na1tzhHs_o.png,https://images2.imgbox.com/4a/80/k1oAkY0k_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1-flight-3-mission-summary,https://en.wikipedia.org/wiki/Trailblazer_(satellite),NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],"Ratsat was carried to orbit on the first successful orbital launch of any privately funded and developed, liquid-propelled carrier rocket, the SpaceX Falcon 1",[],[],[],[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,4,RatSat,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': 

Como se puede ver, muchos de los datos son IDs. Por ejemplo, la columna <code>rocket</code> no tiene información sobre el cohete, solo un número de identificación.

Ahora usar la API nuevamente para obtener información usando los ID proporcionados para cada lanzamiento. Específicamente, usar las columnas de <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, y <code>cores</code>.
Es decir, cohete, cargas útiles, plataforma de lanzamiento y núcleos.

In [13]:
# Tomar un subconjunto del DataFrame solo con las columnas anteriormente mencionadas, más 'flight_number' (número de vuelo), y 'date_utc'.
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# Eliminar las filas con múltiples núcleos porque pertenecen a cohetes Falcon con 2 propulsores adicionales.
# También eliminar filas que tienen múltiples cargas en un solo cohete.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Dado que las cargas y los núcleos son listas de tamaño 1, también extraer el único valor en cada lista y reemplazar la columna por ese valor.
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

'''
la columna 'cores' ahora contiene diccionarios en lugar de listas de diccionarios, y lo mismo la columna 'payloads'. Cada diccionario contiene el único elemento que había en la lista original.
'''

# Convertir 'date_utc' a tipo datetime y luego extraer la fecha, dejando de lado la hora.
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Utilizar la fecha para delimitar las fechas de los lanzamientos.
data = data[data['date'] <= datetime.date(2020, 11, 13)]

+ De <code>rocket</code> obtener el nombre de los propulsores.
+ De <code>payload</code>, obtener la masa de carga útil y la órbita que va a seguir.
+ De <code>launchpad</code>, obtener el nombre del sitio de lanzamiento que se está utilizando, la longitud y la latitud.
+ De <code>cores</code>, obtener el resultado del aterrizaje, el tipo de aterrizaje, el número de vuelos con ese núcleo, si se usaron gridfins, si se reutilizó el núcleo, si se usaron patas, la plataforma de aterrizaje utilizada, el bloque de el núcleo, la cantidad de veces que se ha reutilizado este núcleo específico y el número de serie del núcleo.

Los datos de estas solicitudes se almacenarán en listas que se utilizarán para crear un nuevo DataFrame.

In [14]:
# Recortadorio: Listas / variables
#BoosterVersion = []
#PayloadMass = []
#Orbit = []
#LaunchSite = []
#Outcome = []
#Flights = []
#GridFins = []
#Reused = []
#Legs = []
#LandingPad = []
#Block = []
#ReusedCount = []
#Serial = []
#Longitude = []
#Latitude = []

Las funciones auxiliares anteriormente definidas le agregaran valores a las listas. 

In [15]:
# Llamar la función auxiliar getBoosterVersion.
getBoosterVersion(data)

Ahora la lista BoosterVersion dejó de estar vacía.

In [16]:
# Ver lista actualizada.
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

Aplicar el resto de funciones auxiliares:

In [17]:
# Llamar función getLaunchSite.
getLaunchSite(data)

# Llamar función getPayloadData.
getPayloadData(data)

# Llamar función getCoreData.
getCoreData(data)

Finalmente, construir un dataset utilizando los datos obtenidos. 

In [18]:
# Primero, combinar las columnas en un diccionario.
launch_dict = {
    'FlightNumber': list(data['flight_number']),
    'Date': list(data['date']),
    'BoosterVersion': BoosterVersion,
    'PayloadMass': PayloadMass,
    'Orbit': Orbit,
    'LaunchSite': LaunchSite,
    'Outcome': Outcome,
    'Flights': Flights,
    'GridFins': GridFins,
    'Reused': Reused,
    'Legs': Legs,
    'LandingPad': LandingPad,
    'Block': Block,
    'ReusedCount': ReusedCount,
    'Serial': Serial,
    'Longitude': Longitude,
    'Latitude': Latitude
}

In [19]:
# Luego, convertir el diccionario a DataFrame.
launch_dict_df = pd.DataFrame(launch_dict)

In [20]:
# Mostrar resumen del DataFrame.
launch_dict_df.head()
#launch_dict_df.describe()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857


### Tarea 2: Filtrar el DataFrame para incluir solo los lanzamientos de Falcon 9

Filtrar el DataFrame usando la columna <code>BoosterVersion</code> para mantener solo los lanzamientos de Falcon 9. Guardar los datos filtrados en un nuevo DataFrame llamado data_falcon9.

In [21]:
data_falcon9 = launch_dict_df[launch_dict_df['BoosterVersion'] != 'Falcon 1']

In [22]:
# Debido a la eliminación de algunos valores de la columna 'FlightNumber', se debe volver a establecer esta columna con un nuevo conjunto de valores.
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,86,2020-09-03,Falcon 9,15600.0,VLEO,KSC LC 39A,True ASDS,2,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1060,-80.603956,28.608058
90,87,2020-10-06,Falcon 9,15600.0,VLEO,KSC LC 39A,True ASDS,3,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,13,B1058,-80.603956,28.608058
91,88,2020-10-18,Falcon 9,15600.0,VLEO,KSC LC 39A,True ASDS,6,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1051,-80.603956,28.608058
92,89,2020-10-24,Falcon 9,15600.0,VLEO,CCSFS SLC 40,True ASDS,3,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,12,B1060,-80.577366,28.561857


<strong>Disputa de datos</strong>:

A continuación, se puede ver que a algunas de las filas del dataset les faltan valores.

In [23]:
data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        5
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

El tratamiento de los valores faltantes será el siguiente:<br>
+ La columna <code>LandingPad</code> conservará los valores <code>None</code> para representar cuándo las plataformas de aterrizaje no se usaron.
+ La columna <code>PayloadMass</code> recibirá imputación de datos a través de calcular la media de la columna y reemplazar con esta los valores faltantes que contiene.


### Tarea 3: Tratar los valores faltantes

Calcular la media de <code>PayloadMass</code> usando <code>.mean()</code>. Luego usar la función <code>.replace()</code> para reemplazar los valores <code>np.nan</code> en los datos con la media calculada.

In [24]:
media = data_falcon9['PayloadMass'].mean()
data_falcon9['PayloadMass'] = data_falcon9['PayloadMass'].fillna(media)

In [25]:
# Ver resultados.
data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

Ahora no se deberían tener valores faltantes en el dataset, excepto en <code>LandingPad</code>.

Por último, exportar el dataset a un CSV para la siguiente sección.

In [26]:
data_falcon9.to_csv('../datasets/dataset_part_1.csv', index=False)

> **NOTA:**
Para que las respuestas sean consistentes en caso de que que hayan dificultades, los siguientes archivos utilizarán un dataset proporcionado para que cada cuaderno sea independiente.

## Resumen

El desarrollo de este cuaderno puede resumirse en los siguientes puntos:

+ El primer paso fue Importar las librerías necesarias y definir funciones auxiliares para que la API de Space X extraiga información de sus datos de lanzamiento a partir de IDs. 
+ Luego tuvo lugar la solicitud de datos a la API, los cuales se decodificaron como un objeto Json y convirtieron a un DataFrame de Pandas.
+ El DataFrame dió como resultado muchos IDs que en este punto se utilizaron para extraer información de la API a través de las funciones auxiliares. Para esto, se creó un subconjunto del DataFrame solo con las columnas <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, y <code>cores</code>, más <code>flight_number</code>, y <code>date_utc</code> y se aplicaron modificaciones colaterales. Luego, las funciones auxiliares actualizaron con datos unas listas con nombres correspondientes a las columnas mencionadas, y se las unió en un diccionario que finalmente se convirtió en el DataFrame final.
+ Fue necesario filtrar el DataFrame para incluir solo los lanzamientos del cohete Falcon 9, ya que los datos pertenecientes a este modelo son los necesarios para continuar el proyecto y no otros. También huvo modificaciones colaterales.
+ También se llevó a cabo una disputa de datos para tratar valores faltantes a través de la imputación de valores por media o promedio.
+ Finalmente, importé el dataset final como un archivo CSV para usarlo en la siguiente sección.

---